In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import xarray as xr

import eradiate

eradiate.set_mode("mono")

import eradiate.scenes as ertsc
import eradiate.experiments as ertxp

from eradiate import KernelContext
from eradiate.xarray.interp import dataarray_to_rgb
from eradiate.kernel import UpdateParameter, scene_parameter
from eradiate.units import unit_registry as ureg

In [ ]:
# Create a texture array with RGB values corresponding to land cover types
def landcover_to_rgb(lc_map, colormap):
    texture_buffer = np.full((*lc_map.landcover_class.shape, 3), -1, dtype="float32")

    for lcc in colormap:
        mask = lc_map.landcover_class == lcc
        texture_buffer[mask, :] = colormap[lcc] / 255.0

    return texture_buffer


with open("data/colormap.json", "r") as infile:
    lc_colormap = json.load(infile)
    lc_colormap = {int(k): np.array(v) for k, v in lc_colormap.items()}

lc_map = xr.load_dataset("data/landcover_beijing.nc")
del lc_map.landcover_class.attrs["units"]
lc_map["rgb"] = xr.DataArray(
    landcover_to_rgb(lc_map, lc_colormap),
    {"y": lc_map.y, "x": lc_map.x, "c": ("c", ["r", "g", "b"])},
)
display(lc_map)

# Show land cover types as numbers
lc_map.landcover_class.plot.imshow(aspect=1.15, size=5.5)
plt.show()
plt.close()

# Show land cover types as colours
lc_map.rgb.plot.imshow(aspect=1, size=5)
plt.show()
plt.close()

In [ ]:
# Define basic Experiment parameters
atmosphere = {
    "type": "heterogeneous",
    "molecular_atmosphere": {"type": "molecular"},
    "particle_layers": {
        "type": "particle_layer",
        "bottom": 0 * ureg.km,
        "top": 1 * ureg.km,
        "distribution": "uniform",
        "dataset": "sixsv-urban",
        "tau_ref": 0.3,
        "w_ref": 550 * ureg.nm,
        "has_absorption": True,
    },
}

illumination = {"type": "directional", "zenith": 30, "azimuth": 0}

camera_airplane = {
    "type": "perspective",
    "id": "camera_airplane",
    "origin": [0, 0, 2] * ureg.km,
    "target": [-15, 15, 0] * ureg.km,
    "up": [0, 0, 1],
    "film_resolution": (1280, 720),
    "srf": {"type": "multi_delta", "wavelengths": [440.0, 550.0, 660.0] * ureg.nm},
    "fov": 45,
}

camera_top = {
    "type": "perspective",
    "id": "camera_top",
    "origin": [0, 0, 1900] * ureg.km,
    "target": [0, 0, 0],
    "up": [0, 1, 0],
    "film_resolution": (256, 256),
    "srf": {"type": "multi_delta", "wavelengths": [440.0, 550.0, 660.0] * ureg.nm},
    "fov": 3,
}


@scene_parameter(flags=UpdateParameter.Flags.SPECTRAL)
def lc_rgb(ctx: KernelContext):
    if ctx.si.w == 440.0 * ureg.nm:
        return lc_map.rgb.values[..., [2]]
    elif ctx.si.w == 550.0 * ureg.nm:
        return lc_map.rgb.values[..., [1]]
    elif ctx.si.w == 660.0 * ureg.nm:
        return lc_map.rgb.values[..., [0]]
    else:
        raise RuntimeError(f"Unsupported context: {ctx}")

In [ ]:
# Configure the Experiment
exp = ertxp.AtmosphereExperiment(
    atmosphere=atmosphere,
    illumination=illumination,
    measures=[camera_top, camera_airplane],
    kdict={
        "texture_dem": {
            "type": "bitmap",
            "id": "texture_dem",
            "filter_type": "nearest",
            "wrap_mode": "clamp",
            "data": np.ones_like(lc_map.rgb),
            "raw": True,
        },
        "mat_dem": {
            "type": "diffuse",
            "id": "mat_dem",
            "reflectance": {"type": "ref", "id": "texture_dem"},
        },
        "mesh_dem": {
            "type": "ply",
            "id": "mesh_dem",
            "bsdf": {"type": "ref", "id": "mat_dem"},
            "filename": "data/dem_beijing.ply",
        },
    },
    kpmap={"mat_dem.reflectance.data": lc_rgb},
)
exp.init()

In [ ]:
# Run the simulation
# TODO: Increase sample count
result = eradiate.run(exp, spp=16)

In [ ]:
# Show nadir view
img = dataarray_to_rgb(
    result["camera_top"].radiance, channels=[("w", 660), ("w", 550), ("w", 440)]
)
plt.imshow(img)
plt.axis("off")
plt.show()
plt.close()

In [ ]:
# Show airplane view
img = dataarray_to_rgb(
    result["camera_airplane"].radiance, channels=[("w", 660), ("w", 550), ("w", 440)]
)
plt.imsave("../_images/dem_texturing.png", img)

plt.imshow(img)
plt.axis("off")
plt.show()
plt.close()